### 프림 알고리즘(Prim's Algorithm)
- 대표적인 최소 신장 트리 알고리즘(크루스칼 알고리즘 보다 다소 복잡하다.)
- 프림 알고리즘
    - 시작 정점을 선택한 후, 정점에 인접한 간선 중 최소 간선으로 연결된 정점을 선택하고, 해당 정점에서 다시 최소 간선으로 연결된 정점을 선택하는 방식으로 최소 신장 트리를 확장해가는 방식이다.

* 크루스칼 알고리즘과 프림 알고리즘의 비교
- 둘 다 탐욕 알고리즘을 기초로 하고 있다.(당장 눈 앞의 최소 비용을 선택해서, 결과적으로 최적의 솔루션을 찾음)
- 크루스칼 알고리즘은 가장 가중치가 작은 간선부터 선택하면서 MST 를 구한다.
- 프림 알고리즘은 특정 정점에서 시작하여, 해당 정점에 연결된 가장 가중치가 작은 간선을 선택한다. 간선으로 연결된 정점들에 연결된 간선 중에서 가장 가중치가 작은 간선을 택하는 방식으로 MST 를 구한다.
(가중치 별로 간선 정보를 정렬한 다음, 가장 작은 가중치를 가진 간선부터 선택하는 크루스칼 과는 다르게(그래프 상의 노드의 위치와는 관계없이 간선의 가중치를 기준으로 트리를 만든다.), 특정 노드를 선택해서, 해당 노드에 연결되어 있는 간선들 중, 가장 가중치가 작은 간선을 선택하는 방식이다.)
- 크루스칼의 경우 곳곳에서 선택되어 흩어져 있는 간선들이 마지막엔 하나의 트리로 통합되면서 MST 를 만든다면, 프림의 경우 특정 노드 부터 시작하여 가중치가 작은 간선들을 골라가며 뻗어나가는 방식으로 MST 를 만든다는 차이점이 있다. 

In [4]:
# 프림 알고리즘 코드 작성
# heapq 라이브러리 활용을 통해 우선순위 큐를 사용한다.
import heapq

graph_data = [[2,'A'],[5,'B'],[3,'C']]

heapq.heapify(graph_data) # graph_data 자체가 힙 구조로 변환된다.
    
for index in range(len(graph_data)):
    print(heapq.heappop(graph_data))

[2, 'A']
[3, 'C']
[5, 'B']


In [5]:
# 참고 : Collections 라이브러리의 defaultdict 함수 활용하기
# defaultdict 함수를 사용해서, key 에 대한 value 를 지정하지 않았을 시, 빈 리스트로 초기화한다.
from collections import defaultdict

list_dict = defaultdict(list)
print(list_dict['key1'])

[]


#### 프림 알고리즘 로직
0. 모든 간선 정보를 저장(adjacent_edges)
1. 임의의 정점을 선택, '연결된 노드 집합(connected_nodes)' 에 삽입
2. 선택된 정점에 연결된 간선들을 간선 리스트(candidate_edge_list) 에 삽입
3. 간선 리스트(candidate_edge_list) 에서 최소 가중치를 가지는 간선부터 추출한다.
    - 해당 간선에 연결된 인접 정점이 '연결된 노드 집합' 에 이미 들어 있다면 스킵한다.(cycle 발생을 막기 위함)
    - 해당 간선에 연결된 인접 정점이 '연결된 노드 집합' 에 들어 있지 않으면 해당 간선을 선택하고, 해당 간선 정보를 '최소 신장 트리(mst)' 에 삽입한다.
        - 해당 간선에 연결된 인접 정점의 간선들 중 '연결된 노드 집합(connected_nodes)' 에 없는 노드와 연결된 간선들만 간선 리스트(candidate_edge_list) 에 삽입
        - 연결된 노드 집합(connected_nodes) 에 있는 노드와 연결된 간선들을 간선 리스트에 삽입해도, 해당 간선들은 스킵될 것이기 때문이다.
        - 어차피 스킵될 간선을 간선 리스트(candidate_edge_list) 에 넣지 않음으로 해서, 간선 리스트 에서 최소 가중치를 가지는 간선부터 추출하기 위한 자료구조 유지를 위한 effort 를 줄일 수 있음(예시 : 최소 힙 구조 사용)
4. 선택된 간선은 간선 리스트에서 제거
5. 간선 리스트에 더 이상의 간선이 없을 때까지 3 ~ 4 번을 반복

In [25]:
myedges = [
    (7, 'A', 'B'), (5, 'A', 'D'),
    (8, 'B', 'C'), (9, 'B', 'D'), (7, 'B', 'E'),
    (5, 'C', 'E'),
    (7, 'D', 'E'), (6, 'D', 'F'),
    (8, 'E', 'F'), (9, 'E', 'G'),
    (11, 'F', 'G')
]   

In [26]:
from collections import defaultdict
from heapq import *

def prim(start_node, edges):
    mst = list()
    adjacent_edges = defaultdict(list)
    for weight, n1, n2 in edges:
        adjacent_edges[n1].append((weight, n1, n2))
        adjacent_edges[n2].append((weight, n2, n1))
        
    connected_nodes = set(start_node)
    candidate_edge_list = adjacent_edges[start_node]
    heapify(candidate_edge_list)
    
    while candidate_edge_list:
        weight, n1, n2 = heappop(candidate_edge_list)
        if n2 not in connected_nodes:
            connected_nodes.add(n2)
            mst.append((weight, n1, n2))
            
            for edge in adjacent_edges[n2]:
                if edge[2] not in connected_nodes:
                    heappush(candidate_edge_list, edge)
    
    return mst

In [27]:
prim('A', myedges)

[(5, 'A', 'D'),
 (6, 'D', 'F'),
 (7, 'A', 'B'),
 (7, 'B', 'E'),
 (5, 'E', 'C'),
 (9, 'E', 'G')]